In [1]:
import numpy as np
import pandas as pd

# Cauchy's Steepest Descent

In [3]:
def f(X):
    x1,x2 = X
    return 2*pow(x1,2) - 2*x1*x2 +pow(x2,2)

def gradF(x):
    x1,x2 = x
    return np.array([4*x1 - 2*x2, -2*x1 + 2*x2])

def hessF(x):
    return np.array([[4,-2],[-2,2]])

X=[ np.array([3,5]) ]
alphas = []
grads = []
errs = []
# gradient descent: x(n+1) = x(n) + α ∇f, For Cauchy steep desc. α = argmin f(x(n) + α ∇f)

# for Quadratic function α = argmin f(x(n) + α ∇f(x(n))) => α = - (∇f.T ∇f)/(∇f H ∇f)
# proof : https://ocw.mit.edu/courses/sloan-school-of-management/15-084j-nonlinear-programming-spring-2004/lecture-notes/lec5_steep_desce.pdf

n_iters = 6
for i in range(n_iters):
    gradf_ = gradF(X[-1])
    
    hessf_ = hessF(X[-1])
    # α = (S.T S)/(S.T H S)
    alpha = -1*np.matmul(gradf_.T, gradf_)/(np.matmul(np.matmul(gradf_.T,hessf_),gradf_))
    # Or solve equations usign Scipy. 
    
    X.append(X[-1] + alpha*gradf_)
    
    alphas.append(alpha)
    grads.append(gradf_)
    errs.append(pow(np.linalg.norm(X[-1]-X[-2]),2))
    print("-----Iteration",i+1,"-----")
    print("alpha =",round(alpha,3))
    print("grad = [",round(gradf_[0]),",",round(gradf_[1]),"]")
    print("x1 = ",round(X[-1][0],4),", x2 = " ,round(X[-1][1],4),sep="")
    print("error :",round(errs[-1],7))
    print("\n")

y_vals = [f(x) for x in X]


-----Iteration 1 -----
alpha = -1.25
grad = [ 2 , 4 ]
x1 = 0.5, x2 = 0.0
error : 31.25


-----Iteration 2 -----
alpha = -0.192
grad = [ 2 , -1 ]
x1 = 0.1154, x2 = 0.1923
error : 0.1849112


-----Iteration 3 -----
alpha = -1.25
grad = [ 0 , 0 ]
x1 = 0.0192, x2 = 0.0
error : 0.0462278


-----Iteration 4 -----
alpha = -0.192
grad = [ 0 , 0 ]
x1 = 0.0044, x2 = 0.0074
error : 0.0002735


-----Iteration 5 -----
alpha = -1.25
grad = [ 0 , 0 ]
x1 = 0.0007, x2 = 0.0
error : 6.84e-05


-----Iteration 6 -----
alpha = -0.192
grad = [ 0 , 0 ]
x1 = 0.0002, x2 = 0.0003
error : 4e-07




In [4]:
# Round off and print values
for i,(x1,x2) in enumerate(X):
    X[i][0]=round(x1,3)
    X[i][1]=round(x2,3)
for i,(dx1,dx2) in enumerate(grads):
    grads[i][0] = round(dx1,3)
    grads[i][1] = round(dx2,3)
for i,a in enumerate(alphas):
    alphas[i]=round(a,3)
X = np.array(X)
grads = np.array(grads)

df = pd.DataFrame({"x1":X[:,0][:-1],"x2":X[:,1][:-1],"∂x1":grads[:,0]
                  ,"∂x2":grads[:,1],"α":alphas,"Δx1":X[:,0][1:]-X[:,0][:-1],
                   "Δx2":X[:,1][1:]-X[:,1][:-1],"error":errs,"y_old":y_vals[:-1],"y_new":y_vals[1:]})
df

,x1,x2,∂x1,∂x2,α,Δx1,Δx2,error,y_old,y_new
0,3.000,5.000,2.000,4.000,-1.250,-2.500,-5.000,3.125000e+01,13.000000,5.000000e-01
1,0.500,0.000,2.000,-1.000,-0.192,-0.385,0.192,1.849112e-01,0.500000,1.923077e-02
2,0.115,0.192,0.077,0.154,-1.250,-0.096,-0.192,4.622781e-02,0.019231,7.396450e-04
3,0.019,0.000,0.077,-0.038,-0.192,-0.015,0.007,2.735373e-04,0.000740,2.844788e-05
4,0.004,0.007,0.003,0.006,-1.250,-0.003,-0.007,6.838434e-05,0.000028,1.094149e-06
5,0.001,0.000,0.003,-0.001,-0.192,-0.001,0.000,4.046410e-07,0.000001,4.208267e-08


# Conjugate Gradient Descent

In [98]:
import numpy as np

desc = lambda X,alpha,S: X + alpha*S
lambda_star = lambda X,S,gradf,hessf: -(np.matmul(S.T,gradf(X)))/(np.matmul(np.matmul(S.T,hessf(X)),S))
conj_step = lambda X1,X2,S1,gradf: -gradf(X2)+((np.linalg.norm(gradf(X2))/np.linalg.norm(gradf(X1)))**2)*S1

def conj_grad_desc(X0,gradf,hessf,n_iters):
    X=[X0] #initial guess [0,0]
    S=[]
    errs = []
    # perform one/two step of Cauchy's steepest descent
    for i in range(2):
        S.append(-gradf(X[-1]))
        alpha = lambda_star(X[-1],S[-1],gradf,hessf)
        X.append(desc(X[-1],alpha,S[-1]))
        
        errs.append(pow(np.linalg.norm(X[-1]-X[-2]),2))
        print("Cauchy's Steepest Desc iteration:",i+1)
        print("X_new:",X[-1],"error:",errs[-1])
        print("-------------")
        
    # Conjugate Gradient Descent
    err=[]
    for i in range(n_iters -2):
        S.append(conj_step(X[-2],X[-1],S[-1],gradf))
        if np.linalg.norm(S[-1]) == 0: break
        alpha = lambda_star(X[-1],S[-1],gradf,hessf)
        X.append(desc(X[-1],alpha,S[-1]))
        
        errs.append(pow(np.linalg.norm(X[-1]-X[-2]),2))
        
        print("Conjugate Grad Desc iteration:",i+1)
        print("X_new:",X[-1],"error:",errs[-1])
        print("-------------")
        
    print("Conjugate GD: X =",X[-1])
    return X[-1]
        

def f(X):
    x1,x2 = X
    return 2*pow(x1,2) - 2*x1*x2 +pow(x2,2)

def gradF(x):
    x1,x2 = x
    return np.array([4*x1 - 2*x2, -2*x1 + 2*x2])

def hessF(x):
    return np.array([[4,-2],[-2,2]])

X0=[3,5]
Xf = conj_grad_desc(X0,gradF,hessF,4)
print("MinVal =",f(Xf))

Cauchy's Steepest Desc iteration: 1
X_new: [0.5 0. ] error: 31.250000000000004
-------------
Cauchy's Steepest Desc iteration: 2
X_new: [0.11538462 0.19230769] error: 0.18491124260355027
-------------
Conjugate Grad Desc iteration: 1
X_new: [ 1.11022302e-16 -8.32667268e-17] error: 0.05029585798816569
-------------
Conjugate Grad Desc iteration: 2
X_new: [-5.61348720e-18 -9.04395161e-18] error: 1.911292779552983e-32
-------------
Conjugate GD: X = [-5.61348720e-18 -9.04395161e-18]
MinVal = 4.3279324592976306e-35
